In [1]:
import pandas as pd
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools
import pygsheets as pygs
from datetime import timedelta

In [2]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [3]:
g_drive_cred = r'C:\Users\darius kay\Dev\gitlab\test\python\credentials.json'
workbook = "https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414"

def clear_data_range(workbook_name, tab_name, data_range):
    start_end = str(data_range).split(':')
    start = start_end[0]
    print(start)
    end = start_end[1]
    print(end)
    print(g_drive_cred)
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)
    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)
    # select the first sheet
    wks = ws.worksheet_by_title(tab_name)
    print(ws.id)
    wks.clear(start=start, end=end)
    return

def pd_to_gsheets(workbook_name, sheet_name, df, row_num, col_num):
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)

    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)

    # select the first sheet
    wks = ws.worksheet_by_title(sheet_name)

    # update the designated sheet with df, starting at cell B2.
    wks.set_dataframe(df, (row_num, col_num))

    return

In [4]:
def date_checker(view, week_end):
    if view == 'forecast':
        try:
            return week_end + timedelta(days=21)
        except TypeError:
            return
    else:
        return week_end

In [5]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [21]:
# Primary forecast query.
forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) = 2020"

# Actual primary query.
actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s  HAVING DATE_PART('year', fulfillment_week_ended) = 2020"

In [22]:
# Forecasted qty returned/exchanged query.
forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN send_back__type in %s THEN 1 ELSE 0 END)"

In [23]:
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product"

# Edit for cushion only non-cover.
product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product"

In [24]:
query_params = {
    'send_back_product_weekly': {'view': {'actual': [actual_query, actual_qty], 'forecast': [forecast_query, forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}}
}

In [37]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
#     print(key)
    tab_df = pd.DataFrame()
    combinations = []
    keys = value.keys()
    values = (value[key] for key in keys)
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    for combination in combinations:
        for combo in combination:
            keys = list(keys)
            grouping = ""
            for i in range(len(keys)):
#                 print(query_params[key][keys[i]][combo[keys[i]]])
                if i == 0:
                    query = query_params[key][keys[i]][combo[keys[i]]][0]
#                     print(query)
                    calculation = query_params[key][keys[i]][combo[keys[i]]][1]
#                     print(calculation)
                    params = f"'{combo[keys[i]]}' AS {keys[i]}"
#                     print(params)
                else:
                    grouping = f"{grouping}, {i+1}"
                    if keys[i] == 'send_back__type':
                        calculation = calculation %(str(query_params[key][keys[i]][combo[keys[i]]]))
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
            query = query %(params, calculation, calculation, grouping)
#             print(query)
#             print(calculation)
#             print(params)
#             print(grouping)
            df = pd.read_sql(query, conn)
            tab_df = tab_df.append(df)
#             tab_df = pd.concat([tab_df, df])
    
    # Add 21 days to week_ended for forecast only.
    tab_df['week_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['week_ended']), axis=1)
    
    # Concat first four columns.
    uid = tab_df[['week_ended', 'view', 'product', 'send_back__type']].astype(str).agg('_'.join, axis=1)
    tab_df.insert(loc=0, column='uid', value=uid)

    clear_data_range(workbook, key, "A1:J10000")
    pd_to_gsheets(workbook, key, tab_df, 1, 1)

A1
J10000
C:\Users\darius kay\Dev\gitlab\test\python\credentials.json
1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I


In [36]:
tab_df

,uid,week_ended,view,product,send_back__type,qty_returned,qty_sold,rate
0,2020-09-20_actual_all_send_back_both,2020-09-20,actual,all,send_back_both,1920,32123,0.059770
1,2020-10-18_actual_all_send_back_both,2020-10-18,actual,all,send_back_both,1789,26019,0.068757
2,2020-10-25_actual_all_send_back_both,2020-10-25,actual,all,send_back_both,1763,25705,0.068586
3,2020-11-22_actual_all_send_back_both,2020-11-22,actual,all,send_back_both,1914,32315,0.059229
4,2020-11-29_actual_all_send_back_both,2020-11-29,actual,all,send_back_both,1599,28728,0.055660
...,...,...,...,...,...,...,...,...
112,2020-12-27_forecast_cushioned_exchange,2020-12-27,forecast,cushioned,exchange,0,79,0.000000
113,2021-01-10_forecast_nan_exchange,2021-01-10,forecast,None,exchange,35,4502,0.007774
114,2020-03-08_forecast_classic_exchange,2020-03-08,forecast,classic,exchange,0,12857,0.000000
115,2020-08-09_forecast_nan_exchange,2020-08-09,forecast,None,exchange,27,1717,0.015725


In [31]:
len(uid)

979

In [117]:
# tab_df.loc[tab_df['view'] == 'forecast', 'week_ended_clean'] = tab_df['week_ended'] + timedelta(days=21)
tab_df['week_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['week_ended']), axis=1)

In [118]:
tab_df

,week_ended,view,product,send_back__type,qty_returned,qty_sold,rate,week_ended_clean
0,2019-02-10,actual,all,send_back_both,32,7820,0.004092,2019-02-10
1,2020-09-20,actual,all,send_back_both,1920,32122,0.059772,2020-09-20
2,2018-03-11,actual,all,send_back_both,0,620,0.000000,2018-03-11
3,2019-03-10,actual,all,send_back_both,193,8170,0.023623,2019-03-10
4,2020-10-18,actual,all,send_back_both,1789,26023,0.068747,2020-10-18
...,...,...,...,...,...,...,...,...
260,2016-12-25,forecast,classic,exchange,0,17,0.000000,2016-12-04
261,2017-01-01,forecast,classic,exchange,0,8,0.000000,2016-12-11
262,2017-08-20,forecast,classic,exchange,0,6,0.000000,2017-07-30
263,2017-07-09,forecast,classic,exchange,0,2,0.000000,2017-06-18
